In [2]:
import spacy
from spacy import displacy
import re

In [3]:
nlp = spacy.load('nl_core_news_lg')

In [8]:
import os
import spacy
import re
import chardet
import pandas as pd


input_directory = '/Users/sanam/Desktop/thesis/ONLYPO'
output_directory = '/Users/sanam/Desktop/thesis/tokcpyPO'
ner_output_directory = '/Users/sanam/Desktop/thesis/khodesh/nerPO'
stopwords_path = "/Users/sanam/Desktop/thesis/stopwords-nl.txt"
os.makedirs(output_directory, exist_ok=True)
os.makedirs(ner_output_directory, exist_ok=True)
brin_csv_path = '/Users/sanam/Desktop/thesis/school_data.csv'
quality_df = pd.read_csv(brin_csv_path) 
quality_df = quality_df[['Brin', 'KwaliteiOnderwijs']]
quality_df = quality_df[quality_df['KwaliteiOnderwijs'] != 'Geen eindoordeel']

# Load stopwords from file
with open(stopwords_path, 'r', encoding='utf-8') as file:
    stopwords = set(file.read().split())

for filename in os.listdir(input_directory):
    if filename.endswith(".txt"):
        file_path = os.path.join(input_directory, filename)
        text = None
        
        # Try reading the file with detected encoding
        try:
            brin_value = filename.replace(".txt", "")
            quality_rating = quality_df.loc[quality_df['Brin'] == brin_value, 'KwaliteiOnderwijs']
            if quality_rating.empty or quality_rating.values[0] == 'Geen eindoordeel':
                continue  # Skip this file
            with open(file_path, 'rb') as f:
                rawdata = f.read()
                result = chardet.detect(rawdata)
                encoding = result['encoding']
                confidence = result['confidence']
                
                # If confidence is too low, default to utf-8
                if encoding is None or confidence < 0.7:
                    print(f"Low confidence in encoding detection for {filename}. Defaulting to utf-8.")
                    encoding = 'utf-8'
            
            with open(file_path, 'r', encoding=encoding, errors='replace') as file:
                text = file.read()
        except UnicodeDecodeError:
            print(f"Skipping {filename} due to encoding issues.")
            continue
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue

        if text:
            # Tokenize the text
            doc = nlp(text)
            tokens = []
            for token in doc:
                # Check if the token is a stopword, punctuation, space, digit, or a number-like token
                if (token.is_stop or token.is_punct or token.is_space or 
                    token.is_digit or token.like_num or token.text.lower() in stopwords or
                    re.match(r'^\d', token.text)):
                    continue
                
                # Exclude tokens that are part of entities related to dates or numbers
                if token.ent_type_ in ["DATE", "CARDINAL", "ORDINAL", "QUANTITY", "PERCENT", "MONEY", "TIME"]:
                    continue

                # Exclude tokens that contain any digit
                if re.search(r'\d', token.text):
                    continue
                
                # Exclude tokens that are not alphabetic or are too short
                if not token.text.isalpha() or len(token.text) < 4:
                    continue
                if token in stopwords:
                    continue
                
                # Lemmatize the token
                lemma = token.lemma_
                
                # Exclude lemmas that are stopwords
                
                tokens.append(lemma)
            
            # Save tokenized text
            if tokens:
                output_file_path = os.path.join(output_directory, f"{filename}")
                with open(output_file_path, 'w', encoding='utf-8') as file:
                    file.write('\n'.join(tokens))
            else:
                print(f"No valid tokens were found in {filename}.")

print("Tokenization and NER complete for all files.")


Low confidence in encoding detection for 28BC.txt. Defaulting to utf-8.
Tokenization and NER complete for all files.
